# Project Description

### TODO
#### We are using a CNN, max pooling, and n-grams (a sequence of n words in a sentence) to construct this project

## Imporant Documentation

### PyTorch Resources:
#####    - https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
#####    - https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
#####    - https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html
#####    - https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html
#####    - https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
#####    - https://towardsdatascience.com/understanding-word-n-grams-and-n-gram-probability-in-natural-language-processing-9d9eef0fa058#:~:text=An%20N%2Dgram%20means%20a,3%2Dgram%20(trigram).

### Step 1: Tokenize data and build vocab

In [1]:
import re
import nltk
import torch
import numpy as np
import pandas as pd
import math

ModuleNotFoundError: No module named 'torch'

In [ ]:
class Preprocessing:
    def __init__(self, vocab_size, seq_len):
        self.file_name = 'clickbait_data.csv'
        self.vocab_size = vocab_size
        self.seq_len = seq_len
        self.vocab = None
        self.x_tokenized = None
        self.x_padded = None
        self.x = None
        self.y = None

        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None

    def load_dataset(self):
        # Load dataset from local directory 
        df = pd.read_csv(self.file_name)
        
        self.x = df['headline'].values
        self.y = df['clickbait'].values

    def clean_data(self):
        # Clean data by removing all special characters. Convert words to lowercase
        self.x = [re.sub(r'\'','', headline).lower() for headline in self.x]
        self.x = [re.sub(r'[^A-Za-z0-9]+',' ', headline).lower() for headline in self.x]

    def tokenization(self):
        # Tokenize all headlines
        self.x = [nltk.tokenize.wordpunct_tokenize(headline) for headline in self.x]

    def build_vocab(self):
        # Build vocab and return 'vocab_size' most common words
        self.vocab = dict()

        fdist = nltk.FreqDist()
        for headline in self.x:
            for word in headline:
                fdist[word] += 1
        
        common_words = fdist.most_common(self.vocab_size)

        for count, word in enumerate(common_words):
            self.vocab[word[0]] = count+1
    
    def word_to_idx(self):	
        # Convert each token into index based representation 
        self.x_tokenized = list()
        
        for sentence in self.x:
            temp = list()
            for word in sentence:
                if word in self.vocab.keys():
                    temp.append(self.vocab[word])
            self.x_tokenized.append(temp)
    
    def padding_sentences(self):
        # Make all sentences equal length. 
        # If sentence is smaller than minimum length, pad it 
        idx = 0
        self.x_padded = list()

        for sentence in self.x_tokenized:
            while len(sentence) < self.seq_len:
                sentence.insert(len(sentence), idx)
            self.x_padded.append(sentence)
            
        self.x_padded = np.array(self.x_padded)

    def split_data(self):
        # Split data into training and testing sets
        trnSize = int(len(self.x) * .8)
        tstSize = int(len(self.x) * .2)
        self.x_train = self.x[:trnSize]
        self.y_train = self.y[:trnSize]
        self.x_test = self.x[trnSize:]
        self.y_test = self.y[trnSize:]


### Step 2: Build the Model

In [ ]:
class HeadlineClassifier(torch.nn.ModuleList):
    def __init__(self, seq_len, num_words, embedding_size, dropout, out_size, stride, filters):
        super(HeadlineClassifier, self).__init__()

        self.seq_len = seq_len
        self.num_words = num_words
        self.embedding_size = embedding_size

        # Dropout (used to reduce chance of overfitting by "dropping" units in neural net). Probability (p) set to 0.25
        self.dropout = torch.nn.Dropout(dropout)

        # Kernel sizes for CNN
        #pass it as params when we change the implmentation. Can keep it like this for now
        self.filters = filters

        # Output size for convolutions
        self.out_size = out_size
        # Number of strides for convolutions
        self.stride = stride

        # Embedding layer (lookup table that stores embeddings of a fixed dictionary and size)
        self.embedding = torch.nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)

        # Convolution layers (each is a 1D convolution over an input)
        self.clayers = [torch.nn.Conv1d(self.seq_len, self.out_size, fltr, self.stride) for fltr in self.filters]
        
            
        # Max pooling layers (each applies 1D max pooling to input) 
        self.poollayers = [torch.nn.MaxPool1d(fltr, self.stride) for fltr in self.filters]

        # Fully connected layer (applies linear transformation to data)
        self.fc = torch.nn.Linear(self.size_of_input(), 1)

    
    def size_of_input(self):

        pout = 0
        for filter in self.filters:
            cout = math.floor(((self.embedding_size + (2 * self.padding) - self.dilation * (filter - 1) - 1) / self.stride) + 1)
            pout += math.floor(((cout - (filter - 1) - 1) / self.stride) + 1)
        
        return pout * self.out_size

    '''Create forward pass of neural network. Consists of mainly ordering the different types of layers'''
    def forward_pass(self, input_X):
        
        x = self.embedding(input_X)

        layerPasses = list()
        for i in range(len(self.filters)):
            temp = self.clayers[i](x)
            #Why not sigmoid?
            temp = torch.relu(temp)
            temp = self.poollayers[i](temp)
            layerPasses.append(temp)

        unn = torch.cat(layerPasses, 2)
        unn = unn.reshape(unn.size(0), -1)

        output = torch.sigmoid(self.dropout(self.fc(unn))).squeeze()
        return output


In [ ]:
class DataClass(torch.utils.data.Dataset):
   def __init__(self, x, y):
      self.x = x
      self.y = y
      
   def __len__(self):
      return len(self.x)
      
   def __getitem__(self, idx):
      return self.x[idx], self.y[idx]

In [ ]:
def train(model, dataset, learning_rate, batch_size, train_data, test_data, numOfIter):
    train_loader = torch.utils.data.Dataloader(train_data, batch_size)
    test_loader = torch.utils.data.Dataloader(test_data, batch_size)

    opt = torch.optim.SGD(model.parameters(), lr = learning_rate)

    for iter in numOfIter:
        model.train()
        train_predictions = []

        for x,y in train_loader:
            y = y.type(torch.FloatTensor)
            
            pred = model(x)

            loss = torch.nn.functional.binary_cross_entropy(pred, y)

            opt.zero_grad()

            loss.backwards()

            opt.step()

            train_predictions += list(pred.detach().numpy())

        # Evaluate model
        model.eval()
        test_predictions = []
        with torch.no_grad():
            for x, y in test_loader:
                pred = model(x)
                test_predictions += list(y.detach().numpy())

        # Get accuracy 
        train_acc = calc_acc(dataset['y_train'], train_predictions)
        test_acc = calc_acc(dataset['y_test'], test_predictions)
                
        print("Iteration: %d, Loss: %.5f, Train Accuracy: %.5f" % (iter+1, loss.item(), train_acc))
        print("Iteration: %d, Loss: %.5f, Test Accuracy: %.5f" % (iter+1, loss.item(), test_acc))
        

In [ ]:
def calc_acc(actual, predictions):
    clickbait = 0
    not_clickbait = 0

    for true, pred in zip(actual, predictions):
        if (pred >= .5) and (true == 1):
            clickbait += 1
        elif (pred < .5) and (true == 0):
            not_clickbait += 1

    return (clickbait + not_clickbait) / len(actual)


In [ ]:
# Initialize instance of Preprocessing class
dataset = Preprocessing(2000, 30)

# Load dataset
dataset.load_dataset()

# Clean and tokenize dataset
dataset.clean_data()
dataset.tokenization()

# Build vocab 
dataset.build_vocab()

# Index words and pad headline sentences
dataset.word_to_idx()
dataset.padding_sentences()

# Split data into training and testing sets
dataset.split_data()

# Print data specifics:
print("Dataset size: ", len(dataset.x))
print("Dataset vocab: ", dataset.vocab)
print("Dataset training size: ", len(dataset.y_train))
print("Dataset testing size: ", len(dataset.x_test))

# Initialize model
seq_len = 30
vocab_size = len(dataset.vocab)
embedding_size = 64
dropout = 0.25
out_size = 32
stride = 2
filters = [2,3,4,5]
model = HeadlineClassifier(seq_len, vocab_size, embedding_size, dropout, out_size, stride, filters)

# Creating testing and training datasets
train_set = DataClass(dataset['x_train'], dataset['y_train'])
test_set = DataClass(dataset['x_test'], dataset['y_test'])

# Train model
learning_rate = 0.01
batch_size = 12
numOfIter = 100
train(model, dataset, learning_rate, batch_size, train_set, test_set, numOfIter)


Dataset size:  32000
Dataset vocab:  {'to': 1, 'in': 2, 'the': 3, 'of': 4, 'you': 5, 'a': 6, 'for': 7, 'and': 8, 'on': 9, 'your': 10, 'is': 11, 'are': 12, 'that': 13, 'this': 14, 'with': 15, 'at': 16, 'will': 17, 'from': 18, 'new': 19, 'about': 20, 'what': 21, 'who': 22, 'people': 23, 'things': 24, 'how': 25, 'which': 26, 'us': 27, 'as': 28, 'can': 29, 'by': 30, 'make': 31, 'we': 32, 'know': 33, 'be': 34, 'after': 35, '17': 36, 'do': 37, '21': 38, 's': 39, 'u': 40, 'should': 41, 'have': 42, 'these': 43, 'based': 44, 'all': 45, '19': 46, 'actually': 47, 'up': 48, 'it': 49, 'over': 50, 'their': 51, 'times': 52, 'an': 53, 'out': 54, 'first': 55, 'was': 56, 'its': 57, 'like': 58, 'if': 59, 'one': 60, '2015': 61, 'most': 62, 'or': 63, 'best': 64, 'more': 65, 'life': 66, 'when': 67, 'need': 68, 'heres': 69, 'has': 70, 'time': 71, 'world': 72, 'his': 73, 'just': 74, 'i': 75, '15': 76, 'dead': 77, '23': 78, '18': 79, 'year': 80, 'day': 81, 'her': 82, 'get': 83, 'killed': 84, 'dies': 85, 'ever'

TypeError: 'Preprocessing' object is not subscriptable